<a href="https://colab.research.google.com/github/jsansao/transformers_pt/blob/main/BERTimbau_Fine_tuning_CETENFolha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inferência usando transformers pré-treinados do HuggingFace 

BERTimbau

NOBREAK - Machado

Batch size = 16

GPU T4x2

In [1]:
#@title Passo 1:Instalando Hugging Face Transformers
# We won't need TensorFlow here
!pip uninstall -y tensorflow
# Install `transformers` from master
!pip install transformers datasets
!pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.9.1
# tokenizers version at notebook update --- 0.7.0

Found existing installation: tensorflow 2.6.4
Uninstalling tensorflow-2.6.4:
  Successfully uninstalled tensorflow-2.6.4
tokenizers                            0.12.1
transformers                          4.20.1


In [2]:
#@title Step 1: Loading the Dataset
#1.Load kant.txt using the Colab file manager
#2.Downloading the file from GitHub
!curl -L  https://raw.githubusercontent.com/jsansao/corpus_ptbr/main/dump_Machado_Nobreak.txt --output "dump.txt"
#!curl -L https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_aa --output "dump.txt"
#!curl -L https://raw.githubusercontent.com/jsansao/corpus_ptbr/main/Lyrics_ChicoBuarque.txt --output "kant.txt"

!awk NF < dump.txt > kant.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1739k  100 1739k    0     0      0      0 --:--:-- --:--:-- --:--:--     0   0  3989k      0 --:--:-- --:--:-- --:--:-- 3989k


In [3]:
#@title Passo 2:Baixando e salvando BR_BERTo
#https://huggingface.co/rdenadai/BR_BERTo

from transformers import AutoTokenizer, AutoModelForMaskedLM

#tokenizer = AutoTokenizer.from_pretrained("rdenadai/BR_BERTo")
#model = AutoModelForMaskedLM.from_pretrained("rdenadai/BR_BERTo")

tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
model = AutoModelForMaskedLM.from_pretrained('neuralmind/bert-base-portuguese-cased')

Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
from datasets import load_dataset
#datasets = load_dataset("text", data_files={"train": "kant.txt", "validation": "kant_teste.txt"})
#datasets = load_dataset("text", data_files={"train": "kant.txt"}, split='train[:90%]')
ds = load_dataset("text", data_files={"train": "kant.txt"})

datasets = ds["train"].train_test_split()

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-27e2b3e615bfec30/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [6]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

exemplo de entrada

In [7]:
tokenized_datasets["train"][4]

{'input_ids': [101,
  2010,
  989,
  2196,
  117,
  18968,
  22278,
  146,
  390,
  521,
  11631,
  132,
  253,
  2196,
  106,
  102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [8]:
#block_size = tokenizer.model_max_length
block_size = 32

In [9]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [10]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [11]:
#@title Step 11: Defining a Data Collator
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [12]:
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {metric.compute(predictions=predictions, references=labels)}

In [13]:
#@title Step 12: Initializing the Trainer
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./BERTimbau-Machado_Nobreak",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=16,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["test"],    
)

In [14]:
#@title Step 13: Pre-training the Model
trainer.train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 11066
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 3460
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
500,2.554700
1000,2.306200
1500,2.169300
2000,2.066200
2500,1.985600
3000,1.944000




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=3460, training_loss=2.1345553535946533, metrics={'train_runtime': 956.8325, 'train_samples_per_second': 115.652, 'train_steps_per_second': 3.616, 'total_flos': 1820374818915840.0, 'train_loss': 2.1345553535946533, 'epoch': 10.0})

In [15]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 3751
  Batch size = 16


Perplexity: 8.56


In [16]:
eval_results

{'eval_loss': 2.14743709564209,
 'eval_runtime': 16.6807,
 'eval_samples_per_second': 224.87,
 'eval_steps_per_second': 14.088,
 'epoch': 10.0}

In [17]:
#@title Step 14: Saving the Final Model(+tokenizer + config) to disk
trainer.save_model("./BERTimbau-Machado_Nobreak")

Saving model checkpoint to ./BERTimbau-Machado_Nobreak
Configuration saved in ./BERTimbau-Machado_Nobreak/config.json
Model weights saved in ./BERTimbau-Machado_Nobreak/pytorch_model.bin


In [18]:
# Saving model on Wandb
import wandb
wandb.save('BERTimbau-Machado_Nobreak.h5')

[]

In [19]:
#@title Step 3: Configurando o pipeline fill-mask
#@title Step 15: Language Modeling with the FillMaskPipeline
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./BERTimbau-Machado_Nobreak",
    tokenizer=tokenizer
)

loading configuration file ./BERTimbau-Machado_Nobreak/config.json
Model config BertConfig {
  "_name_or_path": "./BERTimbau-Machado_Nobreak",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 29794
}

loading configuration file ./BERTimbau-Machado_Nobreak/

In [20]:
fill_mask("O rapaz olhou para o [MASK] ")

[{'score': 0.10413004457950592,
  'token': 119,
  'token_str': '.',
  'sequence': 'O rapaz olhou para o.'},
 {'score': 0.05723496899008751,
  'token': 1341,
  'token_str': 'lado',
  'sequence': 'O rapaz olhou para o lado'},
 {'score': 0.05527610704302788,
  'token': 14467,
  'token_str': 'relógio',
  'sequence': 'O rapaz olhou para o relógio'},
 {'score': 0.041138045489788055,
  'token': 1342,
  'token_str': 'outro',
  'sequence': 'O rapaz olhou para o outro'},
 {'score': 0.036971405148506165,
  'token': 21809,
  'token_str': 'cachorro',
  'sequence': 'O rapaz olhou para o cachorro'}]

In [21]:
fill_mask("A moça olhou para o [MASK] ")

[{'score': 0.07597547769546509,
  'token': 14467,
  'token_str': 'relógio',
  'sequence': 'A moça olhou para o relógio'},
 {'score': 0.0605371855199337,
  'token': 1341,
  'token_str': 'lado',
  'sequence': 'A moça olhou para o lado'},
 {'score': 0.05134221911430359,
  'token': 4170,
  'token_str': 'marido',
  'sequence': 'A moça olhou para o marido'},
 {'score': 0.044002778828144073,
  'token': 16909,
  'token_str': 'muro',
  'sequence': 'A moça olhou para o muro'},
 {'score': 0.03901253268122673,
  'token': 119,
  'token_str': '.',
  'sequence': 'A moça olhou para o.'}]

In [22]:
fill_mask("Comprou uma [MASK] na loja. ")

[{'score': 0.2826457917690277,
  'token': 3743,
  'token_str': 'carta',
  'sequence': 'Comprou uma carta na loja.'},
 {'score': 0.04299076274037361,
  'token': 9104,
  'token_str': 'cadeira',
  'sequence': 'Comprou uma cadeira na loja.'},
 {'score': 0.03940936550498009,
  'token': 4428,
  'token_str': 'nota',
  'sequence': 'Comprou uma nota na loja.'},
 {'score': 0.03248007968068123,
  'token': 7924,
  'token_str': 'camisa',
  'sequence': 'Comprou uma camisa na loja.'},
 {'score': 0.03227231279015541,
  'token': 11064,
  'token_str': 'cópia',
  'sequence': 'Comprou uma cópia na loja.'}]

In [23]:
fill_mask("A mulher não é [MASK]. ")

[{'score': 0.14185231924057007,
  'token': 1016,
  'token_str': 'assim',
  'sequence': 'A mulher não é assim.'},
 {'score': 0.046089041978120804,
  'token': 19543,
  'token_str': 'bonita',
  'sequence': 'A mulher não é bonita.'},
 {'score': 0.03142426908016205,
  'token': 2124,
  'token_str': 'forte',
  'sequence': 'A mulher não é forte.'},
 {'score': 0.030483601614832878,
  'token': 19462,
  'token_str': 'santa',
  'sequence': 'A mulher não é santa.'},
 {'score': 0.030340641736984253,
  'token': 3264,
  'token_str': 'boa',
  'sequence': 'A mulher não é boa.'}]

In [24]:
fill_mask("O homem não é [MASK]. ")

[{'score': 0.08181378245353699,
  'token': 1016,
  'token_str': 'assim',
  'sequence': 'O homem não é assim.'},
 {'score': 0.06949321925640106,
  'token': 2397,
  'token_str': 'homem',
  'sequence': 'O homem não é homem.'},
 {'score': 0.061880793422460556,
  'token': 11775,
  'token_str': 'mau',
  'sequence': 'O homem não é mau.'},
 {'score': 0.04127956181764603,
  'token': 13380,
  'token_str': 'perfeito',
  'sequence': 'O homem não é perfeito.'},
 {'score': 0.025666052475571632,
  'token': 4062,
  'token_str': 'bom',
  'sequence': 'O homem não é bom.'}]

In [25]:
fill_mask("Ele é um bom [MASK]. ")

[{'score': 0.30195632576942444,
  'token': 2397,
  'token_str': 'homem',
  'sequence': 'Ele é um bom homem.'},
 {'score': 0.17182405292987823,
  'token': 3695,
  'token_str': 'amigo',
  'sequence': 'Ele é um bom amigo.'},
 {'score': 0.07464156299829483,
  'token': 13254,
  'token_str': 'rapaz',
  'sequence': 'Ele é um bom rapaz.'},
 {'score': 0.042831577360630035,
  'token': 10110,
  'token_str': 'companheiro',
  'sequence': 'Ele é um bom companheiro.'},
 {'score': 0.031354039907455444,
  'token': 21809,
  'token_str': 'cachorro',
  'sequence': 'Ele é um bom cachorro.'}]

In [26]:
fill_mask("Ela é uma boa [MASK]. ")

[{'score': 0.19099988043308258,
  'token': 2760,
  'token_str': 'pessoa',
  'sequence': 'Ela é uma boa pessoa.'},
 {'score': 0.1507546752691269,
  'token': 18073,
  'token_str': 'moça',
  'sequence': 'Ela é uma boa moça.'},
 {'score': 0.11485721170902252,
  'token': 2606,
  'token_str': 'mulher',
  'sequence': 'Ela é uma boa mulher.'},
 {'score': 0.09198703616857529,
  'token': 8932,
  'token_str': 'amiga',
  'sequence': 'Ela é uma boa amiga.'},
 {'score': 0.05999397858977318,
  'token': 17704,
  'token_str': 'senhora',
  'sequence': 'Ela é uma boa senhora.'}]

In [27]:
fill_mask("Faz de conta que ainda é [MASK]. ")

[{'score': 0.675611674785614,
  'token': 1373,
  'token_str': 'tarde',
  'sequence': 'Faz de conta que ainda é tarde.'},
 {'score': 0.09830227494239807,
  'token': 8545,
  'token_str': 'cedo',
  'sequence': 'Faz de conta que ainda é cedo.'},
 {'score': 0.04518957808613777,
  'token': 1695,
  'token_str': 'pouco',
  'sequence': 'Faz de conta que ainda é pouco.'},
 {'score': 0.03625005483627319,
  'token': 596,
  'token_str': 'tempo',
  'sequence': 'Faz de conta que ainda é tempo.'},
 {'score': 0.015292984433472157,
  'token': 2199,
  'token_str': 'possível',
  'sequence': 'Faz de conta que ainda é possível.'}]

In [28]:
fill_mask("Depois da tempestade vem a [MASK]. ")

[{'score': 0.20319883525371552,
  'token': 13943,
  'token_str': 'lua',
  'sequence': 'Depois da tempestade vem a lua.'},
 {'score': 0.11582215875387192,
  'token': 9856,
  'token_str': 'chuva',
  'sequence': 'Depois da tempestade vem a chuva.'},
 {'score': 0.08336257934570312,
  'token': 4527,
  'token_str': 'paz',
  'sequence': 'Depois da tempestade vem a paz.'},
 {'score': 0.07426557689905167,
  'token': 7658,
  'token_str': 'tempestade',
  'sequence': 'Depois da tempestade vem a tempestade.'},
 {'score': 0.06317467242479324,
  'token': 1386,
  'token_str': 'morte',
  'sequence': 'Depois da tempestade vem a morte.'}]

In [29]:
fill_mask("Mais vale um pássaro na mão do que [MASK] voando. ")

[{'score': 0.29421281814575195,
  'token': 1342,
  'token_str': 'outro',
  'sequence': 'Mais vale um pássaro na mão do que outro voando.'},
 {'score': 0.11427430808544159,
  'token': 222,
  'token_str': 'um',
  'sequence': 'Mais vale um pássaro na mão do que um voando.'},
 {'score': 0.09156253933906555,
  'token': 17804,
  'token_str': 'pássaros',
  'sequence': 'Mais vale um pássaro na mão do que pássaros voando.'},
 {'score': 0.07606584578752518,
  'token': 14934,
  'token_str': 'voar',
  'sequence': 'Mais vale um pássaro na mão do que voar voando.'},
 {'score': 0.06261930614709854,
  'token': 368,
  'token_str': 'ele',
  'sequence': 'Mais vale um pássaro na mão do que ele voando.'}]

In [30]:
fill_mask("Não tinha [MASK], mas almoçou mesmo assim. ")

[{'score': 0.7031448483467102,
  'token': 11062,
  'token_str': 'fome',
  'sequence': 'Não tinha fome, mas almoçou mesmo assim.'},
 {'score': 0.047053754329681396,
  'token': 11334,
  'token_str': 'sono',
  'sequence': 'Não tinha sono, mas almoçou mesmo assim.'},
 {'score': 0.014355580322444439,
  'token': 12215,
  'token_str': 'costume',
  'sequence': 'Não tinha costume, mas almoçou mesmo assim.'},
 {'score': 0.014220478013157845,
  'token': 3874,
  'token_str': 'nada',
  'sequence': 'Não tinha nada, mas almoçou mesmo assim.'},
 {'score': 0.013541623018682003,
  'token': 7672,
  'token_str': 'medo',
  'sequence': 'Não tinha medo, mas almoçou mesmo assim.'}]

In [31]:
fill_mask("Bebeu um copo d'água, pois tinha [MASK]. ")

[{'score': 0.4648318886756897,
  'token': 11062,
  'token_str': 'fome',
  'sequence': "Bebeu um copo d'água, pois tinha fome."},
 {'score': 0.20951294898986816,
  'token': 2496,
  'token_str': 'sede',
  'sequence': "Bebeu um copo d'água, pois tinha sede."},
 {'score': 0.1879357248544693,
  'token': 14594,
  'token_str': 'febre',
  'sequence': "Bebeu um copo d'água, pois tinha febre."},
 {'score': 0.020011885091662407,
  'token': 7672,
  'token_str': 'medo',
  'sequence': "Bebeu um copo d'água, pois tinha medo."},
 {'score': 0.019525859504938126,
  'token': 11334,
  'token_str': 'sono',
  'sequence': "Bebeu um copo d'água, pois tinha sono."}]

In [32]:
fill_mask("Sem saber, ele descobriu um [MASK]. ")

[{'score': 0.17505799233913422,
  'token': 11021,
  'token_str': 'segredo',
  'sequence': 'Sem saber, ele descobriu um segredo.'},
 {'score': 0.05680672824382782,
  'token': 16269,
  'token_str': 'mistério',
  'sequence': 'Sem saber, ele descobriu um mistério.'},
 {'score': 0.036026276648044586,
  'token': 3204,
  'token_str': 'plano',
  'sequence': 'Sem saber, ele descobriu um plano.'},
 {'score': 0.03242781013250351,
  'token': 3420,
  'token_str': 'caminho',
  'sequence': 'Sem saber, ele descobriu um caminho.'},
 {'score': 0.03229935094714165,
  'token': 8332,
  'token_str': 'negócio',
  'sequence': 'Sem saber, ele descobriu um negócio.'}]

In [33]:
fill_mask("Pedro fez fortuna vendendo [MASK]. ")

[{'score': 0.1847214251756668,
  'token': 5976,
  'token_str': 'escravos',
  'sequence': 'Pedro fez fortuna vendendo escravos.'},
 {'score': 0.13227835297584534,
  'token': 7568,
  'token_str': 'café',
  'sequence': 'Pedro fez fortuna vendendo café.'},
 {'score': 0.0729813352227211,
  'token': 2978,
  'token_str': 'livros',
  'sequence': 'Pedro fez fortuna vendendo livros.'},
 {'score': 0.04924408718943596,
  'token': 9701,
  'token_str': 'cavalos',
  'sequence': 'Pedro fez fortuna vendendo cavalos.'},
 {'score': 0.04435919225215912,
  'token': 6536,
  'token_str': 'cartas',
  'sequence': 'Pedro fez fortuna vendendo cartas.'}]

In [34]:
fill_mask("Sem medo de ser [MASK]. ")

[{'score': 0.8116620182991028,
  'token': 8540,
  'token_str': 'feliz',
  'sequence': 'Sem medo de ser feliz.'},
 {'score': 0.015370883047580719,
  'token': 3382,
  'token_str': 'visto',
  'sequence': 'Sem medo de ser visto.'},
 {'score': 0.01232471875846386,
  'token': 7148,
  'token_str': 'padre',
  'sequence': 'Sem medo de ser padre.'},
 {'score': 0.009614771232008934,
  'token': 1016,
  'token_str': 'assim',
  'sequence': 'Sem medo de ser assim.'},
 {'score': 0.006740317214280367,
  'token': 8085,
  'token_str': 'descoberto',
  'sequence': 'Sem medo de ser descoberto.'}]

In [35]:
wandb.finish()

eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁▂▃▅▆▇██
train/global_step,▁▂▃▅▆▇██
train/learning_rate,█▇▅▄▂▁
train/loss,█▅▄▂▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
